In [1]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, ConversationHandler, CallbackContext

model = load_model('chatbotmodel.h5')

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

def classify_message(message):
    message = message.lower()
    words_in_message = message.split()
    bag = [0] * len(words)

    for word in words_in_message:
        if word in words:
            bag[words.index(word)] = 1

    p = model.predict(np.array([bag]))
    max_index = np.argmax(p)
    return classes[max_index]

def handle_message(update: Update, context: CallbackContext):
    user_input = update.message.text
    intent = classify_message(user_input)
    
    for intent_data in intents['intents']:
        if intent_data['tag'] == intent:
            responses = intent_data['responses']
            response = random.choice(responses)
            break
    else:
        response = "I'm not sure what you're asking."

    update.message.reply_text(response)

def main():
    global intents
    with open('intents.json') as json_file:
        intents = json.load(json_file)

    updater = Updater(token='6749539229:AAGj1cibmTp0_jKiekJwbfj4ptN0OfT1w6o', use_context=True)
    dp = updater.dispatcher

    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()


2023-10-26 13:17:18.137344: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-26 13:17:18.157168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-26 13:17:18.157185: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-26 13:17:18.157205: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-26 13:17:18.161492: I tensorflow/core/platform/cpu_feature_g

RuntimeError: Bad StatusOr access: INTERNAL: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 25370427392